In [1]:
import numpy as np
import pandas as pd


In [2]:
ligand = pd.read_csv('ligand.pdb', delim_whitespace=True, skiprows=1, header =None, usecols=[2,6,7,8,11], names=['atom_id', 'x', 'y', 'z', 'atom_type'])
ligand_clean = ligand.dropna(thresh=4)
ligand_clean

,atom_id,x,y,z,atom_type
0,C1,41.993,43.168,46.089,C
1,C2,42.560,42.148,46.862,C
2,C3,42.279,41.919,48.194,C
3,C4,41.436,42.842,48.897,C
4,C5,40.785,43.877,48.092,C
5,C6,41.055,43.999,46.645,C
6,Cl1,39.978,45.126,48.946,Cl
7,C7,41.022,42.603,50.344,C
8,O1,41.838,42.514,51.272,O
9,N1,39.630,42.401,50.514,N


In [3]:
positions=ligand_clean[['x','y','z']].to_numpy()
atom_types=ligand_clean['atom_type'].to_numpy()
atom_types
print(positions)

[[41.993 43.168 46.089]
 [42.56  42.148 46.862]
 [42.279 41.919 48.194]
 [41.436 42.842 48.897]
 [40.785 43.877 48.092]
 [41.055 43.999 46.645]
 [39.978 45.126 48.946]
 [41.022 42.603 50.344]
 [41.838 42.514 51.272]
 [39.63  42.401 50.514]
 [39.068 41.945 51.677]
 [39.501 40.762 52.311]
 [38.735 40.116 53.299]
 [37.578 40.654 53.781]
 [37.229 41.905 53.362]
 [37.881 42.551 52.293]
 [35.953 42.448 53.641]
 [34.963 41.901 54.518]
 [35.293 41.036 55.342]
 [43.041 40.453 48.951]
 [42.467 43.206 45.113]
 [43.25  41.431 46.429]
 [40.606 44.824 46.101]
 [39.028 42.622 49.729]
 [40.385 40.202 52.021]
 [38.984 39.138 53.71 ]
 [37.339 43.373 51.835]
 [35.605 43.084 52.933]
 [33.538 42.641 54.295]
 [33.078 41.654 53.153]
 [31.715 41.809 53.965]
 [32.574 41.863 55.267]
 [33.007 42.031 52.127]
 [33.534 40.66  53.211]
 [31.227 42.767 53.758]
 [31.017 40.969 53.891]
 [32.135 42.418 56.102]
 [32.922 40.876 55.59 ]
 [33.568 43.719 54.103]]


In [4]:
atom_dictionary = {'H':1, 'C':6, 'N':7, 'O':8, 'F':9, 'P':15, 'S':16, 'Cl':17}

In [5]:
atoms = np.array([atom_dictionary[letter] for letter in atom_types])
atoms

array([ 6,  6,  6,  6,  6,  6, 17,  6,  8,  7,  6,  6,  6,  7,  6,  6,  7,
        6,  8, 17,  1,  1,  1,  1,  1,  1,  1,  1,  6,  6,  6,  6,  1,  1,
        1,  1,  1,  1,  1])

In [6]:
import torch
import torchani

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchani.models.ANI2x(periodic_table_index=True).to(device)

In [8]:
coordinates = torch.tensor([positions],
                           requires_grad=True, device=device).float()
species = torch.tensor([atoms], device=device)

In [9]:
energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative

In [14]:
#self_energies = torchani.
#e2_energy = torchani.EnergyShifter(energy)
#print(e2_energy)
self_energies = torchani.EnergyShifter.sae(species)

AttributeError: module 'torchani' has no attribute 'fit_intercept'

In [14]:
print('Energy:', energy.item())
print('Force:', force.squeeze())

Energy: -1891.5614290132103
Force: tensor([[ 0.0318, -0.0417,  0.0218],
        [ 0.0132, -0.0343,  0.0087],
        [-0.0118,  0.0196,  0.0297],
        [-0.0450,  0.0033, -0.0505],
        [ 0.0944, -0.0663, -0.0449],
        [-0.0177,  0.0223,  0.1058],
        [-0.0216,  0.0147, -0.0089],
        [ 0.0213, -0.0220,  0.0491],
        [-0.0618,  0.0054, -0.0628],
        [ 0.0754,  0.0392, -0.0484],
        [-0.1005, -0.0251,  0.1194],
        [ 0.0023, -0.0310,  0.0110],
        [-0.0576,  0.0848, -0.0078],
        [ 0.0172,  0.0382,  0.0083],
        [ 0.0136, -0.0729, -0.0752],
        [ 0.0648, -0.0471, -0.0268],
        [-0.0382, -0.0263,  0.0451],
        [ 0.0986,  0.0449,  0.0419],
        [-0.0718,  0.0754, -0.0664],
        [-0.0325,  0.0572, -0.0178],
        [-0.0293,  0.0216, -0.0122],
        [ 0.0026,  0.0067, -0.0127],
        [-0.0028, -0.0021,  0.0011],
        [ 0.0007,  0.0028,  0.0085],
        [ 0.0100,  0.0162, -0.0171],
        [ 0.0062,  0.0085, -0.0056],
   

In [15]:
eshift = torchani.EnergyShifter()

TypeError: __init__() missing 1 required positional argument: 'self_energies'